In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units

from sounding_utils import *

import sys
sys.path.append('../../') # lets us import ptype package from the subdir

#import ptype.

## Select forecast hour:

In [ ]:
hour = '0000'

In [ ]:
rap = xr.open_mfdataset(f"/glade/campaign/cisl/aiml/ptype/ptype_case_studies/kentucky/rap/20220223/{hour}/*.nc", concat_dim='time', combine='nested')
rap.attrs['nwp'] = 'rap'
hrrr = xr.open_mfdataset(f"/glade/campaign/cisl/aiml/ptype/ptype_case_studies/kentucky/hrrr/20220223/{hour}/*.nc", concat_dim='time', combine='nested')
hrrr.attrs['nwp'] = 'hrrr'
gfs = xr.open_mfdataset(f"/glade/campaign/cisl/aiml/ptype/ptype_case_studies/kentucky/gfs/20220223/{hour}/*.nc", concat_dim='time', combine='nested')
gfs.attrs['nwp'] = 'gfs'


In [ ]:
gfs

**Recall our calibration plots:**

![alt text](soundings_files/MLCalibration.png "Calibration of ML model")

In [ ]:
def composites_multiplot(datasets: list[xr.Dataset], y_col: str, 
                              cols: list[str], 
                              colors: list[list[str]], 
                              markers: list[str],
                              prob: float = 0.0) -> None:
    '''function to output composite soundings with shaded quantile regions corresponding to 25/75
       and 10/90 quantiles. input list of predictions to plot and corresponding colors'''

    
    fig = plt.figure(figsize=(9, 9))
    ax = plt.gca()
    
    for i, ds in enumerate(datasets):
        y = ds[y_col].values
        marker = markers[i]
        
        for predtype,c in zip(cols, colors):
            if 'ML' in predtype:
                subset = ds.where((ds['crain'] == 1) | (ds['csnow'] == 1) | (ds['cicep'] == 1) | (ds['cfrzr'] == 1))
                subset = subset.where((ds[predtype] > prob))
            else: 
                subset = ds.where(ds[predtype]==1)
            
            temperature_profile = subset['t_h'].mean(dim=('x','y','time'))
            line, = ax.plot(temperature_profile, y, c, linewidth=2, marker=marker, markersize=8)
            
            #label with num of observations
            num_obs = subset['t_h'].count(dim=('x','y','time')).values[0]
            
            line.set_label((
                f"{ds.attrs['nwp']}: "
                f"{predtype}{'>' + str(prob) if ('ML' in predtype) and (prob >0.0) else ''} |"
                f" Num Obs: {num_obs}"
            ))
    ax.axvline(x=0, ymin=0, ymax=1, c='0')
    ax.set_title(f"Height above Ground and interp. Temp when models predicts {cols[0][1:]}")
    ax.set_xlabel('Temperature (C)')
    ax.set_ylabel('Height above ground (m)')
    ax.legend()

In [ ]:
composites_multiplot([rap, hrrr, gfs], 'heightAboveGround', ['cicep', 'ML_icep'], ['blue', 'orange'], ['o','v', '+'], prob = 0.7)


In [ ]:
composites_multiplot([rap, hrrr, gfs], 'heightAboveGround', ['cfrzr', 'ML_cfrzr'], ['blue', 'orange'], ['o','v', '+'], prob=0.7)


In [ ]:
composites_multiplot([rap, hrrr, gfs], 'heightAboveGround', ['csnow', 'ML_csnow'], ['blue', 'orange'], ['o','v', '+'], prob=0.7)

In [ ]:

composites_multiplot([rap, hrrr, gfs], 'heightAboveGround', ['crain', 'ML_crain'], ['blue', 'orange'], ['o','v', '+'], prob=0.7)

In [ ]:
u, c = np.unique(rap.ML_cicep.values, return_counts=True)